<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/intro_analytics_python/main/images/PA%20Banner.png" width="1000" align="center"></a>


# Chat Bot con Inteligencia artificial
## Procesamiento de lenguaje natural (NLP)

Programa creado para entrear y crear un bot para nuestra aplicación web\
v1.0

In [ ]:
# La última versión de spacy-stanza (>1.0) es compatible solo con spacy >=3.0
# Nota: spacy 3.0 incorpora al pepiline nlp transformers
!pip install -U spacy==3.1 --quiet
!pip install -U spacy-stanza==1.0.0 --quiet

     |████████████████████████████████| 6.4 MB 26.4 MB/s 
     |████████████████████████████████| 621 kB 41.4 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 456 kB 46.0 MB/s 
     |████████████████████████████████| 10.1 MB 29.8 MB/s 
     |████████████████████████████████| 342 kB 35.0 MB/s 


In [ ]:
import json
import string
import random 
import numpy as np

import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
import stanza
import spacy_stanza

stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")


2021-08-25 19:46:42 INFO: Downloading default packages for language: es (Spanish)...


2021-08-25 19:48:32 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-08-25 19:48:32 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-08-25 19:48:33 INFO: Use device: cpu
2021-08-25 19:48:33 INFO: Loading: tokenize
2021-08-25 19:48:33 INFO: Loading: mwt
2021-08-25 19:48:33 INFO: Loading: pos
2021-08-25 19:48:33 INFO: Loading: lemma
2021-08-25 19:48:33 INFO: Loading: depparse
2021-08-25 19:48:33 INFO: Loading: ner
2021-08-25 19:48:35 INFO: Done loading processors!


In [ ]:
import re
import unicodedata

def preprocess_clean_text(text):    
    # remove accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # remove special characters
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    # remove numbers
    text = re.sub(pattern, '', text)
    # remove punctuation
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

### Como funciona el proceso de tokenización y lematización

In [ ]:
text = "personas Ideas! estás cosas muercielagos"

doc = nlp(preprocess_clean_text(text.lower()))
print("tokens:", doc)
for token in doc:
    print(token.lemma_)

tokens: personas ideas estas cosas muercielagos
persona
idea
este
cosa
muercielago


In [ ]:
doc

personas ideas estas cosas muercielagos

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
# used a dictionary to represent an intents JSON file
data = {"intents": [
             {"tag": "bienvenida",
              "patterns": ["Hola", "¿Cómo estás?", "¿Qué tal?"],
              "responses": ["Hola!", "Hola, ¿Cómo estás?"],
             },
             {"tag": "nombre",
              "patterns": ["¿Cúal es tu nombre?", "¿Quién sos?"],
              "responses": ["Mi nombre es MarvelBOT", "Yo soy MarvelBOT"]
             },
            {"tag": "contacto",
              "patterns": ["contacto", "número de contacto", "número de teléfono", "número de whatsapp", "whatsapp"],
              "responses": ["Podes contactarnos al siguiente número +54-9-11-2154-4777", "Contactonos al whatsapp número +54-9-11-2154-4777"]
             },
            {"tag": "envios",
              "patterns": ["¿Realizan envios?", "¿Cómo me llega el paquete?"],
              "responses": ["Los envios se realizan por correo, lo enviaremos a la dirección que registraste en la página"]
             },
            {"tag": "precios",
              "patterns": ["precio", "Me podrás pasar los precios", "¿Cuánto vale?", "¿Cuánto sale?"],
              "responses": ["En el catálogo podrás encontrar los precios de todos nuestros productos en stock"]
             },
            {"tag": "pagos",
              "patterns": ["medios de pago", "tarjeta de crédito", "tarjetas", "cuotas"],
              "responses": ["Contactanos al whatsapp número +54-9-11-2154-4777 para conocer los beneficios y formas de pago vigentes"]
             },
            {"tag": "stock",
              "patterns": ["Esto está disponible", "¿Tenes stock?", "¿Hay stock?"],
              "responses": ["Los productos publicados están en stock"]
             },
            {"tag": "agradecimientos",
              "patterns": [ "Muchas gracias", "Gracias"],
              "responses": ["Por nada!, cualquier otra consulta podes escribirnos"]
             },
             {"tag": "despedida",
              "patterns": [ "Chau", "Hasta luego!"],
              "responses": ["Hasta luego!", "Hablamos luego!"]
             }
]}

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
words = []
classes = []
doc_X = []
doc_y = []
# Tokenizar cada "pattern" y agregar cada palabra al vocabulario (vocabulary)
# Los tokens que se toman de cada pattern se agrega a doc_X
# Cada tag se agrega a doc_y
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        # convert pattern to lowercase and clean it
        tokens = nlp(preprocess_clean_text(pattern.lower()))
        # lemmatize all the words
        for token in tokens:            
            words.append(token.lemma_)
        
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
print("words:", words)
print("classes:", classes)
print("doc_X:", doc_X)
print("doc_y:", doc_y)

words: ['chau', 'como', 'contacto', 'credito', 'cual', 'cuanto', 'cuota', 'de', 'disponible', 'el', 'envio', 'este', 'gracias', 'haber', 'hasta', 'holar', 'llegar', 'luego', 'medio', 'mucho', 'nombre', 'numero', 'pago', 'paquete', 'pasar', 'poder', 'precio', 'que', 'quien', 'realizar', 'salir', 'ser', 'stock', 'tal', 'tarjeta', 'telefono', 'tener', 'tu', 'valer', 'whatsapp', 'yo']
classes: ['agradecimientos', 'bienvenida', 'contacto', 'despedida', 'envios', 'nombre', 'pagos', 'precios', 'stock']
doc_X: ['Hola', '¿Cómo estás?', '¿Qué tal?', '¿Cúal es tu nombre?', '¿Quién sos?', 'contacto', 'número de contacto', 'número de teléfono', 'número de whatsapp', 'whatsapp', '¿Realizan envios?', '¿Cómo me llega el paquete?', 'precio', 'Me podrás pasar los precios', '¿Cuánto vale?', '¿Cuánto sale?', 'medios de pago', 'tarjeta de crédito', 'tarjetas', 'cuotas', 'Esto está disponible', '¿Tenes stock?', '¿Hay stock?', 'Muchas gracias', 'Gracias', 'Chau', 'Hasta luego!']
doc_y: ['bienvenida', 'bienve

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

In [ ]:
# Preparar los datos para entrenamiento
training = []
out_empty = [0] * len(classes)
# Crear la bolsa de palabras (bag of words - bow)
for idx, doc in enumerate(doc_X):
    # Transformar la pregunta (input) en tokens y lematizar
    text = []
    tokens = nlp(preprocess_clean_text(doc.lower()))
    for token in tokens:
        text.append(token.lemma_)

    # Transformar los tokens en "Bag of words" (arrays de 1 y 0)
    bow = []
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    
    # Crear el array de salida (class output) correspondiente
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1

    print("X:", bow, "y:", output_row)
    training.append([bow, output_row])

# Mezclar los datos
random.shuffle(training)
training = np.array(training, dtype=object)
# Dividir en datos de entrada y salida
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 1, 0, 0, 0, 0, 0, 0, 0]
X: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 1, 0, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0] y: [0, 1, 0, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0] y: [0, 0, 0, 0, 0, 1, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 0, 0, 0, 1, 0, 0, 0]
X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 1, 0, 0, 0, 0, 0, 0]
X: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# Definir la entrada y la salida
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])

# Modelo sequencial de aprendizaje profundo DNN
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(output_shape, activation = "softmax"))
#adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5376      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 585       
Total params: 14,217
Trainable params: 14,217
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
hist = model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 731ms/step - loss: 2.2876 - accuracy: 0.1111
Epoch 2/200
1/1 [==============================] - 0s 8ms/step - loss: 2.2045 - accuracy: 0.1111
Epoch 3/200
1/1 [==============================] - 0s 7ms/step - loss: 2.1866 - accuracy: 0.2222
Epoch 4/200
1/1 [==============================] - 0s 7ms/step - loss: 2.2152 - accuracy: 0.1481
Epoch 5/200
1/1 [==============================] - 0s 14ms/step - loss: 2.2355 - accuracy: 0.0370
Epoch 6/200
1/1 [==============================] - 0s 5ms/step - loss: 2.1834 - accuracy: 0.1481
Epoch 7/200
1/1 [==============================] - 0s 5ms/step - loss: 2.1990 - accuracy: 0.1111
Epoch 8/200
1/1 [==============================] - 0s 5ms/step - loss: 2.1941 - accuracy: 0.1111
Epoch 9/200
1/1 [==============================] - 0s 6ms/step - loss: 2.1892 - accuracy: 0.1852
Epoch 10/200
1/1 [==============================] - 0s 4ms/step - loss: 2.1666 - accuracy: 0.1481
Epoch 11/200
1/1 [========

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
def text_to_tokens(text): 
    lemma_tokens = []
    tokens = nlp(preprocess_clean_text(text.lower()))
    for token in tokens:
        lemma_tokens.append(token.lemma_)
    #print(lemma_tokens)
    return lemma_tokens

def bag_of_words(text, vocab): 
    tokens = text_to_tokens(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    #print(bow)
    return np.array(bow)

def pred_class(text, vocab, labels): 
    bow = bag_of_words(text, vocab)
    words_recognized = sum(bow)

    return_list = []
    if words_recognized > 0:
        result = model.predict(np.array([bow]))[0]
        thresh = 0.2
        y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
        y_pred.sort(key=lambda x: x[1], reverse=True)
    
        for r in y_pred:
            return_list.append(labels[r[0]])
            #print(labels[r[0]], r[1])

    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
        if i["tag"] == tag:
            result = "BOT: " + random.choice(i["responses"])
            break
    return result

In [ ]:
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    if len(intents) > 0:
        result = get_response(intents, data)
        print(result)
    else:
        print("Perdón, no comprendo la pregunta.")
    

Hola!
BOT: Hola!


KeyboardInterrupt: ignored

### Descargar el modelo

In [ ]:
# Exportar los datos importants (vocabulario, clases, el bot y el dataset utilizado)
import pickle
import json

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))
model.save('bot.h5')
with open("dataset.json", 'w') fo:
    json.dump(data, fo)

In [ ]:
# Comprimir todos los datos necesarios
!zip -r bot_data.zip classes.pkl words.pkl bot.h5 dataset.json

  adding: bot_model/ (stored 0%)
  adding: bot_model/1/ (stored 0%)
  adding: bot_model/1/assets/ (stored 0%)
  adding: bot_model/1/saved_model.pb (deflated 88%)
  adding: bot_model/1/keras_metadata.pb (deflated 88%)
  adding: bot_model/1/variables/ (stored 0%)
  adding: bot_model/1/variables/variables.data-00000-of-00001 (deflated 18%)
  adding: bot_model/1/variables/variables.index (deflated 62%)
  adding: classes.pkl (deflated 21%)
  adding: words.pkl (deflated 42%)
  adding: bot.h5 (deflated 29%)


In [ ]:
from google.colab import files
files.download('bot_data.zip') 